written by Can Erozer

For finding each of the average value, I will use 30 samples. This sample size is not huge since I don't want to spend a lot of money just on this calculation.

## Finding the mean cost of scraping

In [3]:
import requests
from bs4 import BeautifulSoup

In [28]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.schema import StrOutputParser
from tqdm import tqdm

In [ ]:
api_key=""


These urls are picked randomly from the Raids data and search_results - Abnormal Arrest Days, All Days All Counties.xlsx

In [6]:
urls=["https://www.cbs58.com/news/34-wisconsin-men-were-arrested-by-immigration-and-customs-enforcement",\
     "https://abc7chicago.com/ice-raid-los-angeles-southern-california-four-day-record-244-criminal-immigrants/964411/",\
     "https://www.justice.gov/opa/pr/fourteen-alleged-gang-members-and-associates-indicted-charleston-south-carolina-federal",\
     "https://www.hjnews.com/education/cache-high-senior-works-stays-positive-despite-dads-deportation/article_96d1bd75-0ac0-5ed5-a885-cad032bee9ea.html",\
     "https://www.noozhawk.com/12_arrested_40_in_custody_after_santa_maria_police_led_sweep/",\
     "https://www.azcentral.com/story/noticias/2016/06/17/arrestan-39-inmigrantes-durante-redada-en-wisconsin/86054218/",\
     "https://www.ice.gov/news/releases/more-100-arrested-los-angeles-area-ice-operation-targeting-convicted-criminal-aliens",\
     "https://www.dnainfo.com/chicago/20160811/avondale/this-is-wrong-latino-day-laborer-declares-of-ice-raids-at-job-sites/",\
     "https://www.fox8live.com/story/33050506/undocumented-immigrants-arrested-for-operating-new-orleans-sex-brothel/",\
     "https://weartv.com/news/local/investigators-man-shared-child-pornography-on-kid-friendly-kik-app",\
     "https://www.justice.gov/usao-nh/pr/45-individuals-indicted-participating-fentanyl-trafficking-conspiracy",\
     "https://www.nj.com/news/2018/04/ice_arrests_60_in_nj_in_5-day_enforcement_operatio.html",\
     "https://www.latimes.com/opinion/readersreact/la-le-0819-wednesday-immigration-deport-20150819-story.html",\
    "https://www.safepassageproject.org/2018/02/safe-passage-founder-lenni-benson-comments-on-ice-arrests/",\
    "https://link.springer.com/article/10.1007/s41134-016-0018-8",\
    "https://www.cbsnews.com/news/one-mans-mission-to-rescue-child-sex-trafficking-victims/",\
    "https://www.latimes.com/nation/nationnow/la-na-houston-immigrant-clinic-arrest-20150914-story.html",\
    "https://santaclara.courts.ca.gov/divisions/civil-division",\
    "https://www.newsweek.com/why-did-immigration-and-customs-enforcement-detain-us-citizen-3-and-half-years-282509",\
    "https://www.cbsnews.com/colorado/news/immigration-deportation-michael-hancock/",\
    "https://finbaroneill.blogspot.com/2014/12/jail-release-could-depend-on-prediction.html",\
    "https://www.kcbd.com/story/14664183/immigration-raid-in-odessa-targeted-mcdougal-construction-site/",\
    "https://www.nytimes.com/2016/10/05/us/us-immigration-judges-bias.html",\
    "https://hightimes.com/culture/federal-raids-in-colorado-tell-larger-story/",\
    "https://www.cbp.gov/border-security/along-us-borders/border-patrol-sectors/rio-grande-valley-sector-texas/mcallen-station",\
    "https://www.cnn.com/2017/02/18/politics/kelly-guidance-on-immigration-and-border-security/index.html",\
    "https://france-amerique.com/300-years-of-french-culture-in-alabama/",\
    "https://www.univision.com/univision-news/opinion/the-story-of-the-la-dad-detained-by-ice-after-dropping-daughter-at-school",\
    "https://www.npr.org/sections/thetwo-way/2018/01/31/582307323/feds-formalize-plan-to-pursue-certain-undocumented-immigrants-in-courthouses",\
    "https://www.buzzfeednews.com/article/jasonwells/ice-arrests-up-since-trump"]


In [7]:
len(urls)

30

In [23]:
prompt_extract_info="""I have scraped the text of an article, but it contains extra information like side titles, navigation menus, and other irrelevant content. I also have the title of the article. Please extract only the relevant parts of the text that match the main topic of the article based on its title. Ignore anything unrelated to the topic. 

Here is the contex: {context}

Here is the title: {title}

Instructions:

Focus only on the parts of the text that relate directly to the title and the main topic of the article. If the title is provided as None, focus only on the parts of the text that relate directly to the main topic of the article.
Don't forget to include the important dates in the article. Also, include the publication date of the article if it is present in the text.
Remove any information that is clearly unrelated, like website headers, footers, links, or side titles.
Provide the cleaned-up and relevant text as the output.



Output format:

A cleaned and concise version of the article text, free of irrelevant information. Don't make any explanations."""


In [24]:
#this returns a tuple(str, str). The first element of the tuple is the title of the article
#and the second element of the tuple is the all of the text present in the url
def get_all_texts_from_url(article_urls):
    
    all_texts=[]
    
    for url in article_urls:
    
        # Fetch the webpage content
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML
            soup = BeautifulSoup(response.text, 'html.parser')

            # Get all visible text from the webpage
            article_text = soup.get_text(strip=True)
            
            # Extract text from h1, h2, and p tags
            headings_and_paragraphs = []
            for tag in soup.find_all(['h1']):
                headings_and_paragraphs.append(tag.get_text(strip=True))
                
            if len(headings_and_paragraphs)==0:
                for tag in soup.find_all(['h2']):
                    headings_and_paragraphs.append(tag.get_text(strip=True))
                
            if len(headings_and_paragraphs)==0:
                print("couldn't find the title")
                
                all_texts +=[(None, article_text)]
                
            else:
                
                all_texts +=[(headings_and_paragraphs[0], article_text)]
                

        else:
            print(f"Failed to fetch the webpage. Status code: {response.status_code}")

    
    
    return all_texts
    
    

In [34]:
def get_only_relevant_text(title_text_pairs, model_name):
    
    main_text=[]
    cost=0
    
    for title, context in tqdm(title_text_pairs):        
    
        PROMPT0 = PromptTemplate(template=prompt_extract_info, input_variables=["context", "title"])

        with get_openai_callback() as cb0:
            llm = LLMChain(
                llm = ChatOpenAI(openai_api_key=api_key,
                         temperature=0.01, model=model_name), prompt=PROMPT0)

            response0 = llm.predict(context=context, title=title)
            
            print(f"cost for this scraping:{cb0.total_cost}")

            cost +=cb0.total_cost
            
            print("scraped!")


        main_text +=[title + response0]
        
        
    return main_text, cost

        
    

In [51]:
def write_pred_texts(pred_texts, out_file_names):
    
    directory_out="/Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/text_extraction_experiment/"
    
    file_names=[]
    
    for i in range(len(pred_texts)):
        
        name = "pred_" + out_file_names[i]
        file_names +=[out_file_names[i]]
     
    #for name in out_file_names:
        #name = "pred_" + name
        #file_names +=[name]
        
    #if len(file_names)!=len(pred_texts):
        #print("It seems that some articles did not scraped!")
        
        
    for i in range(len(file_names)):

        with open(directory_out+file_names[i], "w",encoding='utf-8') as file:

            file.write(pred_texts[i])
            print(f"{file_names[i]} written to {directory_out}")
        
    

In [52]:
def pipeline(urls, test_files):
    
    
    results=get_all_texts_from_url(urls)
    
    cleaned_texts, cost=get_only_relevant_text(results, "gpt-4o-mini")            
        
    write_pred_texts(cleaned_texts, test_files)
    
    print(f"Total cost is {cost}")

In [50]:
#not used
def pipeline1(urls):
    
    
    results=get_all_texts_from_url(urls) 
    
    cleaned_texts, cost=get_only_relevant_text(results, "gpt-4o-mini")
    
    print(f"Total cost is {cost}")
    
    


In [53]:
test_files=[str(i)+".txt" for i in range(30)]

pipeline(urls, test_files)

Failed to fetch the webpage. Status code: 403
Failed to fetch the webpage. Status code: 403
Failed to fetch the webpage. Status code: 403
Failed to fetch the webpage. Status code: 403
Failed to fetch the webpage. Status code: 403


  4%|█▊                                          | 1/25 [00:05<02:19,  5.80s/it]

cost for this scraping:0.00043259999999999994
scraped!



  8%|███▌                                        | 2/25 [00:10<02:04,  5.40s/it]

cost for this scraping:0.00033389999999999993
scraped!


...

cost for this scraping:0.0005451
scraped!



 96%|█████████████████████████████████████████▎ | 24/25 [03:28<00:07,  7.19s/it]

cost for this scraping:0.00046229999999999996
scraped!


100%|███████████████████████████████████████████| 25/25 [03:32<00:00,  8.49s/it]

cost for this scraping:0.00034125
scraped!
0.txt written to /Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/text_extraction_experiment/
1.txt written to /Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/text_extraction_experiment/
2.txt written to /Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/text_extraction_experiment/
3.txt written to /Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/text_extraction_experiment/
4.txt written to /Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/text_extraction_experiment/
5.txt written to /Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/text_extraction_experiment/
6.txt written to /Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/text_extraction_experiment/
7.txt written to /Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/text_extraction_experiment/
8.txt written to /Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/text_extraction_experiment/
9.txt written to /Users/canerozer/Desktop/BU/FALL2024/DS7

In [55]:
print(f"mean cost of scraping is  {0.01637715/25}")

mean cost of scraping is  0.000655086


## Finding the mean cost of information extraction

In [56]:
prompt_arrestdate0="""Your task is to analyze the given context and determine if the starting date of the U.S. ICE (Immigration and Customs Enforcement) operation is mentioned.

Key details:

The context is a web article about the operation.
The starting date must include the day, month, and year to be considered "mentioned."
The date might be explicitly stated or inferred through time references (e.g., "yesterday," "last week") combined with the article's publication date (if available).

Response instructions:

Respond True if the starting date is mentioned and all three components (day, month, year) can be identified directly or inferred.
Respond False if any component (day, month, year) is missing or cannot be determined.
Do not guess or make up information.
Context: ### {context} ###"""

In [57]:
prompt_arrestdate1="""Your task is to analyze the given context and determine the starting date of the U.S. ICE (Immigration and Customs Enforcement) operation.

Key details:

The context is a web article about the operation.
The starting date might be stated explicitly (e.g., a specific date) or implicitly through time references (e.g., "yesterday," "last week").
Use the article's publication date (if available) to infer the starting date based on time references.
Focus only on identifying the operation's starting date. Ignore other dates mentioned in the article.

Response format:

Only provide the date in the format mm/dd/yy (e.g., 04/15/23).
If the starting date cannot be determined, do not guess or infer.
Context: ### {context} ###
"""

In [60]:
def check_format(response: str)->bool:
    
    print(f"RESPONSE IS {response}")

    if response[0] not in "01":
        return False
    if response[1] not in "0123456789":
        return False
    if response[2]!="/":
        return False
    if response[3] not in "0123":
        return False
    if response[4] not in "0123456789":
        return False    
    if response[5]!="/":
        return False
    if response[6]!="1":
        return False 
    if response[7] not in "45678":
        return False
    else:
        return True
    
    
def model1_arrestdate(model_name: str, info: str, info_name: str, prompt0: str, prompt1: str):
    
    cost=0.0
    
    PROMPT0 = PromptTemplate(template=prompt0, input_variables=["context"])
    
    with get_openai_callback() as cb0:
        llm = LLMChain(
            llm = ChatOpenAI(openai_api_key=api_key,
                     temperature=0.01, model=model_name), prompt=PROMPT0)
    
        response0 = llm.predict(context=info)
    
        cost +=cb0.total_cost
        print(f"cost for checking is {cb0.total_cost}")
    
    if response0!="True" and response0!="False":
        print("ERROR: Got an unexpected response when looking if the answer is present in the article")
        print(f"UNEXPECTED RESPONSE: {response0}")
        print("The article is:")
        print(info)
        print(f"The article name is {info_name}")
        return None, cost
    
    if response0=="False":
        print("The information you are asking cannot be found only by looking at the given article")
        print("Provide more articles!")
        return None, cost
    
    PROMPT1 = PromptTemplate(template=prompt1, input_variables=["context"])
    
    with get_openai_callback() as cb1:
        llm = LLMChain(
            llm = ChatOpenAI(openai_api_key=api_key,
                     temperature=0.01, model="gpt-4o"), prompt=PROMPT1)
        
        
        response1 = llm.predict(context=info)
    
        cost +=cb1.total_cost
        print(f"cost for extraction is {cb0.total_cost}")
    
    if not (check_format(response1)):
        print("The date is found but is not given in the mm/dd/yy format")
        print(f"Instead got: {response1}")
        return None, cost
    
    print(f"FOUND: {response1}")
    
    
    return response1, cost

            
    

In [61]:
def pipline1_arrestdate(directory, files, model_name):
    
    labels=[]
    cost=0.0
    
    for file in files:
        info=""
        with open(directory+file,"r") as f:
            
            for line in f:
                
                info += line
                
                
        response,c= model1_arrestdate(model_name, info, file, prompt_arrestdate0, prompt_arrestdate1)
        
        cost += c
        
        labels+=["None" if response==None else response]
        
        
    return labels, cost
    
    

In [63]:
pred_directory="/Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/text_extraction_experiment/"
pred_files=[str(i)+".txt" for i in range(25)]

labels, cost= pipline1_arrestdate(pred_directory, pred_files, "gpt-4o")

cost for checking is 0.00271
cost for extraction is 0.00271
RESPONSE IS 05/18/15
FOUND: 05/18/15
cost for checking is 0.0024249999999999996
The information you are asking cannot be found only by looking at the given article
Provide more articles!
cost for checking is 0.003865
The information you are asking cannot be found only by looking at the given article
Provide more articles!
cost for checking is 0.005125
The information you are asking cannot be found only by looking at the given article
Provide more articles!
cost for checking is 0.004825
The information you are asking cannot be found only by looking at the given article
Provide more articles!
cost for checking is 0.001965
The information you are asking cannot be found only by looking at the given article
Provide more articles!
cost for checking is 0.003845
cost for extraction is 0.003845
RESPONSE IS 07/18/16
FOUND: 07/18/16
cost for checking is 0.006175
The information you are asking cannot be found only by looking at the given 

In [64]:
print(cost)

0.093985


In [65]:
print(f"mean cost of extraction is  {0.093985/25}")

mean cost of extraction is  0.0037594


## Finding the mean number of articles found for each raid

In [66]:
andy_directory="/Users/canerozer/Desktop/BU/FALL2024/DS701/DS701_Proje/search_results_Abnormal_Arrest_Days_All_Days_All_Counties.csv"


In [67]:
import pandas as pd

In [89]:
df= pd.read_csv(andy_directory)

In [69]:
df.head()

,query,search_pattern,StateCountyFIPS,ST,CountyName,FIPSState,FIPSCounty,arrest_date,title,url,date_published
0,"Immigration raid Alameda, California",pattern1,6001,CA,Alameda,6,1,8/6/2015,"Readers React: A father deported, kids left be...",https://www.latimes.com/opinion/readersreact/l...,2024-11-16T13:57:00.0000000Z
1,"Immigration raid Alameda, California",pattern1,6001,CA,Alameda,6,1,8/6/2015,"Spurned by local law enforcement, ICE stages e...",https://www.latimes.com/local/politics/la-me-i...,2024-10-27T19:58:00.0000000Z
2,"Immigration raid Alameda, California",pattern1,6001,CA,Alameda,6,1,8/6/2015,California gives immigrants here illegally unp...,https://www.latimes.com/local/california/la-me...,2024-11-12T06:30:00.0000000Z
3,"Immigration raid Alameda, California",pattern1,6001,CA,Alameda,6,1,8/6/2015,California's Special Restrictions on Who May C...,https://www.littler.com/publication-press/publ...,2024-11-19T23:54:00.0000000Z
4,"Immigration raid Alameda, California",pattern1,6001,CA,Alameda,6,1,8/6/2015,"UPAC Raids | News, Videos & Articles",https://globalnews.ca/tag/upac-raids/,2024-07-08T20:32:00.0000000Z


In [90]:
result = df.groupby(["StateCountyFIPS","CountyName", "FIPSState","FIPSCounty","arrest_date"], as_index=False).agg({
    "url": lambda x: list(x)  # Combine URLs into a list
})

In [91]:
result.head()

,StateCountyFIPS,CountyName,FIPSState,FIPSCounty,arrest_date,url
0,1081,Lee,1,81,8/18/2016,[https://www.thenation.com/article/archive/adv...
1,1089,Madison,1,89,11/9/2016,[https://www.justice.gov/opa/pr/former-madison...
2,1097,Mobile,1,97,1/10/2018,[https://www.nbcnews.com/news/us-news/immigrat...
3,1097,Mobile,1,97,1/13/2017,[https://academic.oup.com/ije/article/46/3/839...
4,1097,Mobile,1,97,1/18/2018,[https://www.courthousenews.com/ice-finalizes-...


In [92]:
result.shape

(4239, 6)

In [93]:
result["len_urls"]=result["url"].apply(lambda x: len(x))

In [94]:
result.head()

,StateCountyFIPS,CountyName,FIPSState,FIPSCounty,arrest_date,url,len_urls
0,1081,Lee,1,81,8/18/2016,[https://www.thenation.com/article/archive/adv...,40
1,1089,Madison,1,89,11/9/2016,[https://www.justice.gov/opa/pr/former-madison...,40
2,1097,Mobile,1,97,1/10/2018,[https://www.nbcnews.com/news/us-news/immigrat...,40
3,1097,Mobile,1,97,1/13/2017,[https://academic.oup.com/ije/article/46/3/839...,40
4,1097,Mobile,1,97,1/18/2018,[https://www.courthousenews.com/ice-finalizes-...,40


In [95]:
result["len_urls"].mean()

41.152158527954704

In [97]:
print(f"mean number of articles found for each raid is  {result['len_urls'].mean()}")

mean number of articles found for each raid is  41.152158527954704
